In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [3]:
!git clone https://github.com/XuhuiZhou/CATS.git
import sys; sys.path.append('./CATS')

Cloning into 'CATS'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 77 (delta 33), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [4]:
import torch
from scipy.special import softmax
import numpy as np
import math
import pickle

from transformers import GPT2Tokenizer, GPT2LMHeadModel

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model=model.to(device)

In [6]:
from datasets import load_dataset
dataset = load_dataset("wikitext", 'wikitext-2-raw-v1')

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91. Subsequent calls will reuse this data.


In [ ]:
!python run_clm.py \
    --model_name_or_path gpt2 \
    --block_size 100\
    --dataset_name wikitext \
    --dataset_config_name wikitext-2-raw-v1 \
    --do_train \
    --do_eval \
    --output_dir /tmp/test-clm


In [ ]:
model.eval()

In [9]:
 unigram_freq = pickle.load(open('gpt-openwebtext.pickle', "rb"))
 unigram_total = sum(unigram_freq.values()) 

In [10]:
def predict(text, model, tokenizer):
    text = text
    tokenized_text = tokenizer.tokenize(text)
    sentence_score = 0
    tokens_tensor = torch.tensor([[50256] + tokenizer.convert_tokens_to_ids(tokenized_text)], device=device)
    length = len(tokenized_text)

    #unigram logprob
    uni_lp = 0.0
    for w in tokenized_text:
      uni_lp += math.log(float(unigram_freq[w])/unigram_total)

    with torch.no_grad():
        outputs = model(tokens_tensor, labels= tokens_tensor)
    loss, logits  = outputs[:2]
    sentence_score = -loss


    lp = float(loss.item()) * -1.0 * length
    penalty = ((5+length)**0.8 / (5+1)**0.8)*lp

    div_lp = -lp / uni_lp
    slor = (lp - uni_lp) / length
    pen_slor = (lp - uni_lp) / penalty

    return sentence_score, lp, penalty, div_lp, slor, pen_slor

In [73]:
def show_results():
    with open("CATS/commonsense_ability_test/{}.txt".format(test), "r") as f:
        file = f.readlines()
    num = len(file)
    count = 0
    curr = 0
    count_slor = 0
    count_pen_slor = 0

    for line in file:
        print(line)
        line = line.strip().split("\001")
        label = int(line[0])
        score_list = []
        lp_list = []
        div_lp_list = []
        penalty_list = []
        slors_list = []
        pen_slor_list = []
        for sentence in line[1:]:
            score, lp, penalty, div_lp, slor, pen_slor = predict(sentence, model=model, tokenizer=tokenizer)
            score_list.append(score)
            lp_list.append(lp)
            div_lp_list.append(div_lp)
            penalty_list.append(penalty)
            slors_list.append(slor)
            pen_slor_list.append(pen_slor)

        ppl_list = [math.exp(-element) for element in score_list]
        print('scores:', [element.item() for element in score_list])
        print('ppl:', ppl_list)
        print('lp:', lp_list)
        print('div_lp:', div_lp_list)
        print('penalty:', penalty_list)
        print('slor:', slors_list)
        print('pen_slor:',  pen_slor_list, '\n')

   
        predict_label = ppl_list.index(min(ppl_list))
        print('predicted label ppl:', predict_label, '  correct label:', label)
        if predict_label==label:
            count += 1
        curr += 1
        print ('correct answers:', count, '  total number of examples:', curr, '  percentage of correct answers:', count/curr, '\n')

        predict_label_slore = slors_list.index(max(slors_list))
        print('predicted label slor:', predict_label_slore, '  correct label:', label)
        if predict_label_slore==label:
            count_slor += 1
        print ('correct answers:', count_slor, '  total number of examples:', curr, '  percentage of correct answers:', count_slor/curr, '\n')

        predict_label_pen_slore = pen_slor_list.index(min(pen_slor_list))
        print('predicted label pen slor:', predict_label_pen_slore, '  correct label:', label)
        if predict_label_pen_slore==label:
            count_pen_slor += 1
        print ('correct answers:', count_pen_slor, '  total number of examples:', curr, '  percentage of correct answers:', count_pen_slor/curr, '\n')


    print (' percentage of correct answers ppl:', count/num)
    print (' percentage of correct answers slor:', count_slor/num)
    print (' percentage of correct answers pen_slor:', count_pen_slor/num)

In [74]:
test = 'smr'
show_results()

Streaming output truncated to the last 5000 lines.
2" a campfire can be nice in a cold tent " is against common sense because we don't need campfire if we have air conditioning . " a campfire can be nice in a cold tent " is against common sense because we don't need campfire if we have lights . " a campfire can be nice in a cold tent " is against common sense because the campfire will burn the tent . 

scores: [-4.364541053771973, -4.6141037940979, -4.525090217590332]
ppl: [78.61331237809607, 100.89736318835261, 92.30425292030301]
lp: [-130.93623161315918, -133.8090100288391, -113.1272554397583]
div_lp: [-0.5352401751502415, -0.5724589728174823, -0.5569078500626368]
penalty: [-536.776226329329, -535.9786505774761, -409.9616707520439]
slor: [3.789818907243378, 3.4460437678986873, 3.6002939318365144]
pen_slor: [-0.21180999016067856, -0.18645382453459539, -0.21955064269984348] 

predicted label ppl: 0   correct label: 2
correct answers: 702   total number of examples: 1759   percentage

In [75]:
test = 'ca'
show_results()

0The trophy doesn't fit into the brown suitcase because the trophy is too large.The trophy doesn't fit into the brown suitcase and the trophy is too large.

scores: [-3.7938218116760254, -3.8596951961517334]
ppl: [44.42586352534108, 47.45088595176637]
lp: [-60.701148986816406, -61.755123138427734]
div_lp: [-0.4590139699963686, -0.4783520303447394]
penalty: [-165.36795173376305, -168.2392902428603]
slor: [4.471333629466734, 4.209038605124542]
pen_slor: [-0.4326191219121281, -0.4002906668518273] 

predicted label ppl: 0   correct label: 0
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

predicted label slor: 0   correct label: 0
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

predicted label pen slor: 0   correct label: 0
correct answers: 1   total number of examples: 1   percentage of correct answers: 1.0 

1The trophy doesn't fit into the brown suitcase and the suitcase is too small.The trophy doesn't fi

In [76]:
test = 'arct_1'
show_results()

Streaming output truncated to the last 5000 lines.
0People want to discuss issues rather than be told what to think, and since the people discussing issues in the comments section know how to discuss, Comment sections have not failedPeople want to discuss issues rather than be told what to think, but since the people discussing issues in the comments section do not know how to discuss, Comment sections have not failed

scores: [-3.7877023220062256, -3.6564998626708984]
ppl: [44.15483005150555, 38.72556060140722]
lp: [-124.99417662620544, -127.97749519348145]
div_lp: [-0.4955538104028748, -0.47743328331868806]
penalty: [-547.2624584097832, -583.7953057703185]
slor: [3.8556700877970687, 4.002161547053598]
pen_slor: [-0.23249742594627187, -0.23993967194040036] 

predicted label ppl: 1   correct label: 0
correct answers: 87   total number of examples: 182   percentage of correct answers: 0.47802197802197804 

predicted label slor: 1   correct label: 0
correct answers: 92   total number o

In [77]:
test = 'arct_2'
show_results()

Streaming output truncated to the last 5000 lines.
1People want to discuss issues rather than be told what to think, and since the people discussing issues in the comments section know how to discuss, Comment sections have failedPeople want to discuss issues rather than be told what to think, but since the people discussing issues in the comments section do not know how to discuss, Comment sections have failed

scores: [-3.797646999359131, -3.5511887073516846]
ppl: [44.59612622709656, 34.85472493983169]
lp: [-121.52470397949219, -120.74041604995728]
div_lp: [-0.49428738334857586, -0.4614043767892952]
penalty: [-520.8407442389071, -539.7384447899402]
slor: [3.885427922828386, 4.145289449320352]
pen_slor: [-0.23871729488482013, -0.2611261855392644] 

predicted label ppl: 1   correct label: 1
correct answers: 313   total number of examples: 626   percentage of correct answers: 0.5 

predicted label slor: 1   correct label: 1
correct answers: 324   total number of examples: 626   percent